# Introduction to Tools in AutoGen AgentChat

## Overview

Language Models (LLMs) are powerful, but their capabilities are often limited to generating text. To enable agents to perform real-world actions, interact with external systems, or access up-to-date information, they need the ability to use **tools**.

In AutoGen AgentChat, tools are functions or external services that an agent can call to extend its capabilities. This notebook introduces the fundamental concepts of tools, how to define them, and how to integrate them with an `AssistantAgent`.

A tool is essentially a callable function or a subclass of `autogen_core.tools.BaseTool`. The `AssistantAgent` automatically converts Python functions into `FunctionTool` instances, generating the necessary schema from function signatures and docstrings. Crucially, when an `AssistantAgent` executes a tool, it does so directly within the same call to its `run()` method.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

## Defining a Simple Tool (Python Function)

The simplest way to define a tool is by writing a standard Python function. AutoGen's `AssistantAgent` can automatically convert this function into a callable tool for the LLM.

In [2]:
import asyncio
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# Define a tool using a Python function
def greet(name: str) -> str:
    """Return a friendly greeting for the user."""
    return f"Hello, {name}! Nice to meet you."

# Wrap it into a FunctionTool (this step is often implicit with AssistantAgent)
greet_tool = FunctionTool(greet, description="Return a friendly greeting for a given name.")

async def main():
    # Create the OpenAI model client
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

    # Create the agent, passing the tool
    agent = AssistantAgent(
        name="greeter",
        model_client=model_client,
        tools=[greet_tool],
        system_message="You are a helpful assistant that can greet people by name."
    )

    print("\n🔄 Task: Greet Alice.")
    print("-" * 40)
    # Ask the agent to use the tool
    await Console(agent.run_stream(task="Greet someone named Alice."))

    # Cleanup
    await model_client.close()

await main()


🔄 Task: Greet Alice.
----------------------------------------
---------- TextMessage (user) ----------
Greet someone named Alice.
---------- ToolCallRequestEvent (greeter) ----------
[FunctionCall(id='call_bS8hidtLLcWSodIkswMUW4UR', arguments='{"name":"Alice"}', name='greet')]
---------- ToolCallExecutionEvent (greeter) ----------
[FunctionExecutionResult(content='Hello, Alice! Nice to meet you.', name='greet', call_id='call_bS8hidtLLcWSodIkswMUW4UR', is_error=False)]
---------- ToolCallSummaryMessage (greeter) ----------
Hello, Alice! Nice to meet you.


## Tool Output Reflection (`reflect_on_tool_use`)

By default, when an `AssistantAgent` executes a tool, it will return the tool's raw output as a string in a `ToolCallSummaryMessage`. However, if your tool's output is not a well-formed natural language string (e.g., it's a complex JSON object, a raw data dump, or just a boolean), you might want the model to *reflect* on the tool's output and summarize it in natural language.

You can enable this reflection step by setting the `reflect_on_tool_use=True` parameter in the `AssistantAgent` constructor.

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# A tool that returns a non-natural language output (e.g., JSON string)
def get_system_status() -> str:
    """Returns the current system status as a JSON string."""
    return '{"cpu_usage": "85%", "memory_free": "2GB", "service_status": "running"}'

async def run_reflection_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

    # Agent with reflection enabled
    agent_with_reflection = AssistantAgent(
        name="status_reporter",
        model_client=model_client,
        tools=[get_system_status],
        system_message="You are an assistant that reports system status. Summarize the tool output.",
        reflect_on_tool_use=True # Enable reflection
    )

    print("\n🔄 Task (with reflection): Report system status.")
    print("-" * 40)
    await Console(agent_with_reflection.run_stream(task="What is the current system status?"))

    # Agent without reflection (for comparison)
    agent_no_reflection = AssistantAgent(
        name="raw_reporter",
        model_client=model_client,
        tools=[get_system_status],
        system_message="You are an assistant that reports system status. Just output the raw tool result.",
        reflect_on_tool_use=False # Explicitly disable reflection
    )

    print("\n🔄 Task (without reflection): Report system status.")
    print("-" * 40)
    await Console(agent_no_reflection.run_stream(task="What is the current system status?"))

    await model_client.close()

await run_reflection_example()


🔄 Task (with reflection): Report system status.
----------------------------------------
---------- TextMessage (user) ----------
What is the current system status?
---------- ToolCallRequestEvent (status_reporter) ----------
[FunctionCall(id='call_QMb8zVffeVBIVjJDvPZTj91q', arguments='{}', name='get_system_status')]
---------- ToolCallExecutionEvent (status_reporter) ----------
[FunctionExecutionResult(content='{"cpu_usage": "85%", "memory_free": "2GB", "service_status": "running"}', name='get_system_status', call_id='call_QMb8zVffeVBIVjJDvPZTj91q', is_error=False)]
---------- TextMessage (status_reporter) ----------
The current system status is as follows:
- CPU Usage: 85%
- Free Memory: 2GB
- Service Status: Running

🔄 Task (without reflection): Report system status.
----------------------------------------
---------- TextMessage (user) ----------
What is the current system status?
---------- ToolCallRequestEvent (raw_reporter) ----------
[FunctionCall(id='call_RNBqeDLcReSLmQ2mySRh

## Next Steps

This notebook covered the basics of defining and using tools, along with the `reflect_on_tool_use` parameter. To continue learning:

1.  **Explore Built-in Tools**: Check out `08_Built_in_Tools.ipynb` for a variety of pre-built tools in AutoGen Extension.
2.  **Advanced Tool Usage**: Dive into `10_Advanced_Tool_Usage.ipynb` to learn about parallel tool calls and tool iterations.
3.  **Custom Tools**: Implement more complex custom tools that interact with external APIs or databases.
4.  **Tool Validation**: Explore how to add input and output validation to your tools using Pydantic.